In [ ]:
!python -m spacy download nb_core_news_sm > /dev/null

In [1]:
import os
import pandas as pd
import spacy
from spacy.lang.nb.examples import sentences 
from nltk.tokenize import word_tokenize
import re
import string
from nltk.corpus import stopwords
import fasttext
from nltk import NaiveBayesClassifier
from nltk import classify
import random
import tqdm
from itertools import groupby
import pickle
import numpy as np
from sklearn.metrics import r2_score

In [2]:
def filter_split_dataset(metadata_df = None):
    """Filter out nynorsk, and split into train and test datasets.
    
    :param metadata_df: pd.DataFrame object with metadata.
    
    :return: 2-tuple of train and test pd.DataFrames
    """
    if metadata_df is None:
        metadata_df = pd.read_json(f"data/metadata.json").T
        
    bokmål_mask = metadata_df.language == "nb"
    
    metadata_df = metadata_df[bokmål_mask]
    
    train_mask = metadata_df.split == "train"
    test_mask = metadata_df.split == "test"
        
    return metadata_df[train_mask], metadata_df[test_mask]
    

In [3]:
def process_documents(doc, lemmatizer=None, remove_newlines=False):
    """Tokenize and lemmatize.
    
    :param document: String.
    :param remove_newlines: Bool, whether to remove newline characters.
    :param lemmatizer: Function Str -> iter(Str,)
    
    :return: [Spacy Doc? object,]
    """
    
    assert isinstance(doc, str), f"doc has to be of type str, {type(doc)} is not supported."
    
    if lemmatizer is None:
        lemmatizer = spacy.load("nb_core_news_sm")
    
    if remove_newlines:
        doc = re.sub('\n', '', doc)
    
    return lemmatizer(doc)

In [4]:
def clean_document(doc, stop_words, **process_kwargs):
    """Cleans up document, normalising words and removing stop_words.
    
    :param doc: [Str]
    :param stop_words: [Str] to exclude
    :param **process_kwargs: Passed to process_documents
    
    :return: [[Str]], list of tokens for each doc
    """
    full_list = []
    for token in process_documents(doc, **process_kwargs):
        if token.lemma_ not in stop_words:
            if token.lemma_ in string.punctuation:
                full_list.append(token.lemma_)
            else:
                full_list.append(f" {token.lemma_}")
    
    return full_list

In [5]:
def get_documents(metadata_df=None, path="data", ret=["rating", "authors"]):
    """Get documents of a specific type.
        
    :param path: Str path to folder with test and train folders.
    :param dataset: Determines which type to look for. Either `train` or `test`.
    :param ret: Columns from each document to return alongside the document itself. [Rating and authors]
    
    :return: iter(Str,) of documents
    """
    
    if metadata_df is None:
        metadata_df = pd.read_json(f"{path}/metadata.json").T
    
    full_path = f"{path}/%s/%s.txt"
    
    for (_, review) in metadata_df.iterrows():
        document = open(f"{path}/{review['split']}/{str(review['id']).zfill(6)}.txt", "r").read()
        yield document, *[review[col] for col in ret]

In [6]:
def make_processed_datasets(debug=False, **datasets):
    """Make fasttext-style dataset, with each line being a text.
    
    :param debug: Set to true to only process first doc.
    :param **datasets: pd.DataFrames of the metadata format
    
    Create files `../data/processed/<kwarg_key>.txt`, with each line
    being on the form `__label__<1-6> <a document, without linebreaks>.
    
    :return: None
    """
    for split, dataset in datasets.items():
        num_docs = len(dataset)
        
        text_list = []
        file = open(f'../data/processed/{split}.txt', 'a')
        for doc, rating in tqdm.tqdm(get_documents(path="../data", metadata_df = dataset, ret=["rating"]), total=len(dataset)):
            
            clean_str = "".join(clean_document(doc, stop_words=stop_words, lemmatizer=lemmatizer, remove_newlines=True))
            text_list.append(f"__label__{rating} {clean_str}\n")
            
            if debug:
                break
                    
        random.shuffle(text_list)
        
        with open(f"../data/processed/{split}.txt", "w") as file:
            file.write("\n".join(text_list))
        

In [7]:
metadata_df = pd.read_json(f"../data/metadata.json").T

In [25]:
def set_gender(authors, authors_df):
    """Gives gender of a list of authors based on labeled names in a df.
    
    :param authors: [Str,] where each element is an author
    :param authors_df: pd.DataFrame with names as index and a column with gender info.
    
    :return: `m` for male, `k` for female, or `u` for unknown or ambigous.
    """
    if len(authors) == 1:
        return authors_df.gender[authors[0]]
    else:
        gender = authors_df.gender[authors[0]]
        for author in authors[1:]:
            if gender != authors_df.gender[author]:
                return "u"
        return gender

def label_author_gender(metadata_df):
    """Finds all author names, and prompts user to label their gender.
    
    Stores author genders in `../data/authors.csv`.
    
    :param metadata_df: pd.DataFrame
    
    :return: metadata_df with gender column.
    """
    
    all_authors = []
    for authors in metadata_df.authors:
        all_authors.extend(authors)
    all_authors = list(set(all_authors))

    first_names = list(set([full_name.split()[0] for full_name in all_authors]))

    genders = {}
    i = 0
    while i < len(first_names):
        gender = input(f"Sex of `{first_names[i]}` ({i}/{len(first_names)})`: ")
        if gender.lower() in ["m", "k", "u"]:
            genders[first_names[i]] = gender.lower()
            i += 1
        elif gender.lower() == "r":
            i -= 1
            print("Correcting error, type last gender again.")
        else:
            print("Type either `m` for male, `k` for female, or `u` for unknown/other.")

    authors_df = pd.DataFrame(index=all_authors, data={"name": all_authors})

    def apply_firstname_gender(full_name):
        return genders[full_name.split()[0]]

    authors_df["gender"] = authors_df.name.apply(apply_firstname_gender)
    authors_df.to_csv("../data/authors.csv")

    metadata_df["gender"] = metadata_df.authors.apply(lambda authors: set_gender(authors, authors_df))
    
    return metadata_df

if os.path.exists("../data/authors.csv") or "gender" in metadata_df.columns:
    overwrite = input("Vil du overskrive dataen om kjønnet til forfattere? [Y/n] ").lower()
    
    if overwrite in ["y", "yes"]:
        metadata_df = label_author_gender(metadata_df)
    
    if not "gender" in metadata_df.columns:
        authors_df = pd.read_csv("../data/authors.csv", index_col="name")
        metadata_df["gender"] = metadata_df.authors.apply(lambda authors: set_gender(authors, authors_df))
else:
    metadata_df = label_author_gender(metadata_df)

Vil du overskrive dataen om kjønnet til forfattere? [Y/n]  n


In [26]:
metadata_df["rating"] = metadata_df["rating"].astype(int)
metadata_df.groupby("gender")["rating"].mean()

gender
k    4.143918
m    4.153312
u    4.211404
Name: rating, dtype: float64

In [10]:
train_metadata_df, test_metadata_df = filter_split_dataset(metadata_df = metadata_df)
datasets = {'train': train_metadata_df, 'test': test_metadata_df}
lemmatizer = spacy.load("nb_core_news_sm")

In [11]:
# Extract stop words in bokmål from http://snowball.tartarus.org/algorithms/norwegian/stop.txt
stop_words = []
with open("../data/stop_words.txt", "r") as stop_words_file:
    for line in stop_words_file:
        if len(line) >= 2 and line[2] != "|":
            stop_word, explanation, = line.split("|")
            if len(stop_word) > 1 and explanation[-2] != "*":
                stop_words.append(stop_word.strip())
print(stop_words)

['og', 'i', 'jeg', 'det', 'at', 'en', 'et', 'den', 'til', 'er', 'som', 'på', 'de', 'med', 'han', 'av', 'ikke', 'der', 'så', 'var', 'meg', 'seg', 'men', 'ett', 'har', 'om', 'vi', 'min', 'mitt', 'ha', 'hadde', 'hun', 'nå', 'over', 'da', 'ved', 'fra', 'du', 'ut', 'sin', 'dem', 'oss', 'opp', 'man', 'kan', 'hans', 'hvor', 'eller', 'hva', 'skal', 'selv', 'sjøl', 'her', 'alle', 'vil', 'bli', 'ble', 'blitt', 'kunne', 'inn', 'når', 'være', 'kom', 'noen', 'noe', 'ville', 'dere', 'som', 'deres', 'kun', 'ja', 'etter', 'ned', 'skulle', 'denne', 'for', 'deg', 'si', 'sine', 'sitt', 'mot', 'å', 'meget', 'hvorfor', 'dette', 'disse', 'uten', 'hvordan', 'ingen', 'din', 'ditt', 'blir', 'samme', 'hvilken', 'hvilke', 'sånn', 'inni', 'mellom', 'vår', 'hver', 'hvem', 'vors', 'hvis', 'både', 'bare', 'enn', 'fordi', 'før', 'mange', 'også', 'slik', 'vært', 'være', 'begge', 'siden', 'henne', 'hennar', 'hennes']


In [12]:
# Dette lager to datasett, et treningssett og et testsett, for fasttext-modellen
# Vi trenger ikke kjøre dette hver gang, bare første gang vi bruker det på en spesifikk datamaskin
if any (os.path.exists(f"../data/processed/{split}.txt") for split in ["train", "test"]):
    overwrite = input(f"Vil du overskrive de prosesserte datasettetene ditt? [Y/n] ").lower()
    if overwrite in ["y", "yes"]:
        make_processed_datasets(**datasets)
else:
    make_processed_datasets(**datasets)

Vil du overskrive de prosesserte datasettetene ditt? [Y/n]  


In [13]:
model = fasttext.train_supervised(input="../data/processed/train.txt", epoch=30, lr=1.0, wordNgrams=3, verbose=2)

In [14]:
model.save_model("fasttext_model.bin")

In [15]:
model.test("../data/processed/test.txt")

(4281, 0.5676243868255081, 0.5676243868255081)

In [16]:
def predict(doc, model, lemmatizer=None, stop_words=stop_words):
    """Process text, and use the model to predict a label.
    
    :param doc: Str
    :param model: Model with predict method.
    :lemmatizer: Function Str -> iter(Str,)
    """
    if lemmatizer is None:
        lemmatizer = spacy.load("nb_core_news_sm")
    
    clean_doc = "".join(clean_document(doc, stop_words=stop_words, lemmatizer=lemmatizer, remove_newlines=True))
    prediction = model.predict(clean_doc)
    return prediction[0][0][-1], clean_doc

In [17]:
predict("Dette var en fantastisk film, leverte på alle punkter. Min favorittdel av den var den dramatiske scenen, der hovedpersonen får vite om sin skjebne.", model, lemmatizer=lemmatizer)

('4',
 ' fantastisk film, levere punkt. favorittdel dramatisk scene, hovedperson får vite skjebne.')

In [18]:
def sort_by_value(unsorted_dict):
    """Sorts dictionary by its value
    
    :param unsorted_dict: Dict
    
    :return: Sorted dict, descending
    """
    
    return {k: v for k, v in sorted(unsorted_dict.items(), key=lambda item: item[1], reverse=True)}

In [19]:
def sentencize(lemmatizer=None, **datasets):
    """Take in datasets, and return all text, split on sentences.
    
    :param **datasets: pd.DataFrame metadata-style object.
    
    :return: Dict where each dataset has a list of strings, each being a sentence.
    """
    
    if lemmatizer is None:
        lemmatizer = spacy.load("nb_core_news_sm")
    
    ret = {}
    with tqdm.tqdm(total=sum([len(dataset) for _, dataset in datasets.items()])) as pbar:
        for split, dataset in datasets.items():
            ret[split] = []
            for doc, in get_documents(metadata_df=dataset, path="../data", ret=[]):
                pbar.update(1)
                for sentence in process_documents(doc, lemmatizer=lemmatizer, remove_newlines=True).sents:
                    ret[split].append(sentence.text.split())
    return ret
    

In [20]:
def group_words(length, **sentence_sets):
    """Groups words in lists of length == length
        
    :param length: Int. Number of words in each string to return.
    :param **sentence_sets: [Str], where each string is a sentence.
    
    :return: Dictionary with all unique groups, string : int number of occurences.
    """
    word_groups = {}
    with tqdm.tqdm(total = sum([len(sentence_set) for sentence_set in sentence_sets])) as pbar:
        for split, sentence_set in sentence_sets.items():
            pbar.update(1)
            word_groups[split] = []
            for sentence in sentence_set:
                for i in range(len(sentence) - length):
                    word_groups[split].append(" ".join([sentence[i+ii] for ii in range(length)]))
            word_groups[split].sort()
            word_groups[split] = {key: len(list(group)) for key, group in groupby(word_groups[split])}
            word_groups[split] = sort_by_value(word_groups[split])
    return word_groups

In [21]:
def generate_gendered_dataset(dataset):
    """Split dataset based on gender.
    
    :param dataset: pd.DataFrame metadata-style object.
    
    :return: Dict with male, female and unknown pd.DataFrame metadata-style objects
    """
        
    return {gender: dataset[dataset.gender == gender] for gender in ["m", "k", "u"]}

In [22]:
def sentence_complexity(lemmatizer=None, **sentence_sets):
    """Finds the average word count per sentence, and char count per word.
    
    :param lemmatizer:
    :param **sentence_sets:
    
    :return: Dict with same keys as sentence_sets, and 2-tuples with avg word/sent and char/word as values.
    """
    
    if lemmatizer is None:
        lemmatizer = spacy.load("nb_core_news_sm")
        
    complexity = {}
    for split, sentence_set in sentence_sets.items():
        words = 0
        chars = 0
        for sentence in sentence_set:
            words += len(sentence)
            chars += len("".join(sentence))
        
        words_per_sent = words / len(sentence_set)
        chars_per_word = chars / words
        
        complexity[split] = (words_per_sent, chars_per_word)
        
    return complexity

In [23]:
def word_ratios(ratio_threshold, absolute_threshold, **word_groups_dicts):
    """Finds the words that are used most in a group
    
    TODO: Normalise with regard to amount of words written by each group, and in each tag
    
    :param ratio_threshold: Float
    :param absolute_threshold: Int
    :param **word_groups: Dictionary with all unique groups, string : int number of occurences.
    
    :return: Dictionary with the 
    """
    split = list(word_groups_dicts.keys())
    split_combos = [(split[i], split[j]) for j in range(len(split)) for i in range(len(split)) if i != j]
    
    ratios = {}
    for split1, split2 in split_combos:
        ratios[split1 + split2] = {}
        for word_group in word_groups_dicts[split1]:
            if word_group in word_groups_dicts[split2] and word_groups_dicts[split2][word_group] >= absolute_threshold:
                ratio = word_groups_dicts[split1][word_group]/word_groups_dicts[split2][word_group]
                if ratio >= ratio_threshold:
                    ratios[split1 + split2][word_group] = ratio
        
        ratios[split1 + split2] = sort_by_value(ratios[split1 + split2])
    
    return ratios

In [24]:
gender_dataset = generate_gendered_dataset(datasets["train"])

overwrite = "y"
if os.path.exists("../data/sentence_sets.pkl"):
    overwrite = input("Vil du overskrive dataen om setningssammensetning? [Y/n] ").lower()
    
if overwrite in ["y", "yes"]:
    sentence_set = sentencize(lemmatizer=lemmatizer, **gender_dataset)

    with open("../data/sentence_sets.pkl", "wb") as handle:
        pickle.dump(sentence_set, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open("../data/sentence_sets.pkl", "rb") as handle:
        sentence_set = pickle.load(handle)

Vil du overskrive dataen om setningssammensetning? [Y/n]  n


In [27]:
word_groups = [0 for _ in range(5)]
for i in range(1,5):
    print(f"\rWorking on finding word groups of length {i}", flush=True, end="")
    word_groups[i] = group_words(i, **sentence_set)
print("Found word groups for all lengths up to 4")

Working on finding word groups of length 1

100%|██████████| 3/3 [00:17<00:00,  5.79s/it]

Working on finding word groups of length 2


100%|██████████| 3/3 [00:27<00:00,  9.25s/it]

Working on finding word groups of length 3


100%|██████████| 3/3 [00:35<00:00, 11.79s/it]

Working on finding word groups of length 4


100%|██████████| 3/3 [00:36<00:00, 12.01s/it]

Found word groups for all lengths up to 4


In [28]:
complexity = sentence_complexity(lemmatizer=lemmatizer, **sentence_set)

In [29]:
print(complexity) # Ikke signifikant

{'m': (14.401525810052043, 5.1038499318228245), 'k': (14.411139517143269, 5.09687424667392), 'u': (15.272584895537461, 5.011558558532961)}


In [30]:
for i, _word_groups in enumerate(word_groups[1:]):
    ratios = word_ratios(1.5, 10, **_word_groups)
    print(f"Ordgrupper med lengde {i+1}")
    for split_combos, desc in [("km", "Kvinner mer enn menn"), ("mk", "Menn mer enn kvinner")]:
        print(f"   {desc}:")
        for i, (word, ratio) in enumerate(ratios[split_combos].items()):
            if i >= 10:
                break
            print(f"         {word}: {ratio:.1f}")

Ordgrupper med lengde 1
   Kvinner mer enn menn:
         storyen: 7.2
         vurderes: 6.6
         service,: 6.3
         tilskueren: 5.9
         Retten: 5.2
         Antall: 4.9
         Peers: 4.3
         Ferrantes: 4.1
         konsept,: 4.0
         levendegjør: 3.7
   Menn mer enn kvinner:
         BESTE: 52.7
         sjøl: 23.1
         kutt: 20.5
         ANBEFALTE: 14.9
         time,: 14.9
         plater: 14.6
         iPhone: 14.5
         rockens: 14.5
         soloalbum: 13.9
         salg: 13.7
Ordgrupper med lengde 2
   Kvinner mer enn menn:
         1 time: 11.8
         over all: 7.5
         pris og: 7.2
         innenfor sin: 7.1
         Se traileren: 4.1
         Vigdis Hjorth: 4.0
         dem imellom: 3.8
         av ett: 3.5
         Regissøren har: 3.3
         på menyen: 3.1
   Menn mer enn kvinner:
         en sang: 16.7
         Oslo Spektrum: 15.3
         Of The: 14.7
         sånn at: 14.0
         sang om: 12.6
         du finner: 12.5
         og

In [31]:
def eec_generator(templates, people, feelings):
    """Generates corpus with all combinations of templates, people and feelings.
    
    :param templates: [Str]
    :param people: [(Str: id, Str: value)] 
    :param feelings: [(Str: id, Str: value)]
    
    :return: 2-tuple og corpus and ids
    """
    
    corpus = []
    ids = []
    for template in templates:
        for person in people:
            for feeling in feelings:
                corpus.append(template.format(person=person[1], feeling=feeling[1]))
                ids.append((template, person[0], person[0]))
    
    return corpus, ids

In [32]:
templates = ["{person} er {feeling}",
             "Situasjonen får {person} til å bli {feeling}",
             "Jeg fikk {person} til å bli {feeling}"]
people = [("m", "sønnen min"), ("k", "datteren min"), ("m", "han"), ("k", "henne")]
feelings = [("anger", "sint"), ("joy", "glad")]

eec_corpus = eec_generator(templates, people, feelings)

In [33]:
for prompt, identifier in zip(*eec_corpus):
    prediction = predict(prompt, model, lemmatizer=lemmatizer)
    print(f"{prompt}/{prediction[1]}: {prediction[0]}")
# Her ser vi at modellen fungerer dårlig til dette. Muligens delvis pga bias mot korte tekster,
# delvis fordi han og hun er i stop_words, og delvis fordi en film kan være god eller dårlig uavhengig
# av om protagonisten har det godt eller dårlig

sønnen min er sint/ sønn sint: 3
sønnen min er glad/ sønn glad: 3
datteren min er sint/ datter sint: 3
datteren min er glad/ datter glad: 3
han er sint/ sint: 3
han er glad/ glad: 3
henne er sint/ sint: 3
henne er glad/ glad: 3
Situasjonen får sønnen min til å bli sint/ situasjon få sønn sint: 4
Situasjonen får sønnen min til å bli glad/ situasjon få sønn glad: 4
Situasjonen får datteren min til å bli sint/ situasjon få datter sint: 4
Situasjonen får datteren min til å bli glad/ situasjon få datter glad: 4
Situasjonen får han til å bli sint/ situasjon få sint: 4
Situasjonen får han til å bli glad/ situasjon få glad: 4
Situasjonen får henne til å bli sint/ situasjon få sint: 4
Situasjonen får henne til å bli glad/ situasjon få glad: 4
Jeg fikk sønnen min til å bli sint/ få sønn sint: 3
Jeg fikk sønnen min til å bli glad/ få sønn glad: 3
Jeg fikk datteren min til å bli sint/ få datter sint: 3
Jeg fikk datteren min til å bli glad/ få datter glad: 3
Jeg fikk han til å bli sint/ få sint: 3


In [34]:
# Tester at modellen gir andre svar enn 3 og 4, og fungerer litt iallfall
doc = open(f"../data/test/000307.txt", "r").read()
clean = clean_document(doc, stop_words, lemmatizer=lemmatizer, remove_newlines=True)
print(f"Fasit er 4, modellen gjetter {predict(doc, model, lemmatizer=lemmatizer)}[0]")

Fasit er 4, modellen gjetter ('5', ' Bloodline S02 E01- e05familiehemmelighete mørk. Bloodline fjorårets høydepunkt blant dramakrimserie. veldig god sammenskrudd rollegalleri, historie pløye dyp dysfunksjonell familiedynamikk seig intens fortellerstil gjøre nøsting Rayburn-familiens hemmelighet kvalitetsserie. likevel skrike mye sesong 1 ferdig. serie ganske naturlig perfekt slutt. tross sist episode ende liten cliffhange, veldig tilfreds tid tilbrakt Florida. fremdeles mye glede serie fortsette familiesagae. skuespillerprestasjon, atmosfære, god dialog all tid investere karakter gjøre rask glire serieunivers ny. oppleves ta fatt lang seig epilog. stor mysterium drive spenning løse fjor, etterspill nerve. Fortidens løgne brenner under bein kaste handling forlate rayburn-familie sesong. begivenhet slutt forrige sesong skape ny mørk familiehemmelighet ta bra dagslys, bruke hemmelighet egen vinning. par ny familiemedlem ankomme idyllisk strandeiendom, gjøre fortid fortsette kaste skygge m

To analyseretninger herfra:

1) Se om modellen gir bedre, dårligere, eller mer varierende anmeldelser hvis noe handler tydelig om menn enn om kvinner
2) Se om modellen gir bedre, dårligere, eller mer varierende anmeldelser hvis ordene som brukes er mer kvinnelige eller mannlige

Sjekke om fordelingen av ord mellom kjønnene kan være tilfeldig

In [35]:
def test_model_gender_bias(model, **datasets):
    """Test the model on datasets, and find distribution of errors in each of them.
    
    :param model: Fasttext model object.
    :param **datasets: pd.DataFrame metadata-style objects.
    """
    ratings = {}
    total_rows = sum([len(dataset) for (_, dataset) in datasets.items()])
    with tqdm.tqdm(total=total_rows) as pbar:
        for split, dataset in datasets.items():
            ratings[split] = np.zeros((len(dataset), 2))
            for i, (doc, actual_rating) in enumerate(get_documents(path="../data", metadata_df = dataset, ret=["rating"])):
                pbar.update(1)
                predicted_rating, _ = predict(doc, model, lemmatizer=lemmatizer, stop_words=stop_words)
                ratings[split][i] = np.array([actual_rating, predicted_rating])
    
    return ratings

In [36]:
gender_test_datasets = generate_gendered_dataset(datasets["test"])
ratings = test_model_gender_bias(model, **gender_test_datasets)

100%|██████████| 4281/4281 [03:57<00:00, 18.03it/s]


In [37]:
for split, ratings_array in ratings.items():
    r2 = r2_score(ratings_array[:,0], ratings_array[:,1])
    actual_avg = ratings_array[:,0].mean()
    pred_avg = ratings_array[:,1].mean()
    print(f"{split.capitalize()}: R2 = {r2:.3f}. Actual/predicted avg rating = {actual_avg:.2f}/{pred_avg:.2f}")

M: R2 = 0.320. Actual/predicted avg rating = 4.25/4.31
K: R2 = 0.362. Actual/predicted avg rating = 4.23/4.27
U: R2 = 0.363. Actual/predicted avg rating = 4.08/4.07


Her ser vi at modellen ikke har en signifikant bias mot kvinner. Den gjør det også bedre enn baseline-modellene i norec_baselines.

In [57]:
# Denne fungerer dårligere. For mange bindestreker, kolon og anførselstegn som blir ord
def group_words_in_docs(*lengths, **datasets):
    """Groups words in lists of length == length
        
    :param *lengths: Int. Number of words in each string to return.
    :param **datasets: pd.DataFrame metadata-style objects.
    
    :return: Dictionary with all unique groups, string : int number of occurences.
    """
    word_groups = {length: {split: [] for split in datasets.keys()} for length in lengths}
    with tqdm.tqdm(total = sum([len(dataset) for _, dataset in datasets.items()])) as pbar:
        for split, dataset in datasets.items():
            for doc, in get_documents(path="../data", metadata_df = dataset, ret=[]):
                pbar.update(1)
                processed_doc = process_documents(doc, lemmatizer=lemmatizer, remove_newlines=True)
                for length in lengths:
                    for i in range(len(processed_doc) - length):
                        word_groups[length][split].append(" ".join([processed_doc[i+ii].text for ii in range(length)]))
            
            for length in lengths:
                word_groups[length][split].sort()
                word_groups[length][split] = {key: len(list(group)) for key, group in groupby(word_groups[length][split])}
                word_groups[length][split] = sort_by_value(word_groups[length][split])
    
    return word_groups

In [51]:
word_groups2 = group_words_in_docs(1, 2, 3, 4, 5, **{split: gender_dataset[split] for split in ["m", "k"]})

100%|██████████| 27050/27050 [23:16<00:00, 19.37it/s]   


In [53]:
with open("../data/word_groups.pkl", "wb") as handle:
    pickle.dump(word_groups2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [58]:
overwrite = "y"
if os.path.exists("../data/word_groups.pkl"):
    overwrite = input("Vil du overskrive dataen om ordgrupper? [Y/n] ").lower()
    
if overwrite in ["y", "yes"]:
    word_groups = group_words_in_docs(1, 2, 3, 4, 5, **gender_dataset)

    with open("../data/sentence_sets.pkl", "wb") as handle:
        pickle.dump(word_groups, handle, protocol=pickle.HIGHEST_PROTOCOL)
else:
    with open("../data/sentence_sets.pkl", "rb") as handle:
        word_groups = pickle.load(handle)

Vil du overskrive dataen om ordgrupper? [Y/n]  n


In [56]:
for length, _word_groups in word_groups2.items():
    ratios = word_ratios(1.5, 10, **_word_groups)
    print(f"Ordgrupper med lengde {length}")
    for split_combos, desc in [("km", "Kvinner mer enn menn"), ("mk", "Menn mer enn kvinner")]:
        print(f"   {desc}:")
        for i, (word, ratio) in enumerate(ratios[split_combos].items()):
            if i >= 10:
                break
            print(f"         {word}: {ratio:.1f}")

Ordgrupper med lengde 1
   Kvinner mer enn menn:
         TEATER: 11.0
         individuelt: 7.9
         Lengde: 7.3
         oppskrifter: 5.5
         vurderes: 5.3
         Retten: 5.2
         Antall: 4.9
         tilskueren: 4.7
         storyen: 4.4
         Peers: 4.4
   Menn mer enn kvinner:
         LÅT: 55.6
         BESTE: 52.8
         Xbox: 44.0
         _: 37.2
         kutt: 25.6
         Blu-ray: 23.7
         PS: 23.4
         Universal: 22.6
         PC: 20.7
         tittelkuttet: 18.7
Ordgrupper med lengde 2
   Kvinner mer enn menn:
         . Aldersgrense: 8.3
         Lengde :: 7.7
         over all: 7.5
         , mat: 7.4
         pris og: 7.2
         innenfor sin: 7.1
         og atmosfære: 6.9
         . Skuespillere: 6.8
         service ,: 6.3
         min. Aldersgrense: 6.2
   Menn mer enn kvinner:
         LÅT :: 55.6
         BESTE LÅT: 54.2
         . BESTE: 50.6
         . PS: 29.0
         på Twitter: 19.5
         On The: 15.6
         KJØP :: 15.5
 

In [39]:
def find_words_in_docs(words, context_chars=200, max_occurences=3, **datasets):
    """Find examples of a specific string occuring in datasets.
    
    :param words: Str, the words you are interested in seeing.
    :param context_chars: How many charaters to include on either side of the words, in the print. [100]
    :param max_occurences: How many occurences you want to find before stopping. [3]
    :param **datasets: pd.DataFrame metadata-style objects.
    
    Prints out the context as fast as it is found.
    
    :return: Tuple of ids where text was found.
    """
    ids = []
    with tqdm.tqdm(total = sum([len(dataset) for _, dataset in datasets.items()])) as pbar:
        for split, dataset in datasets.items():
            for doc, doc_id, excerpt in get_documents(path="../data", metadata_df = dataset, ret=["id", "excerpt"]):
                pbar.update(1)
                if doc.find(words) != -1:
                    words_idx = doc.find(words)
                    print("---")
                    print(f"\033[1m\033[95m{excerpt}: \033[0m")
                    print("...", doc[max(0, words_idx-context_chars):words_idx], end="")
                    print("\033[1m\033[4m", doc[words_idx:words_idx+len(words)], "\033[0m", end="")
                    print(doc[words_idx+len(words): min(len(doc), words_idx+len(words)+context_chars)])
                    ids.append(doc_id)
                    if len(ids) >= max_occurences:
                        print("---")
                        print("Found max occurences, stopped before searching through entire dataset")
                        return ids
    return ids

In [40]:
find_words_in_docs("kvinner er", context_chars=200, max_occurences=10, **gender_dataset)

  0%|          | 77/34478 [00:00<00:44, 769.83it/s]

---
Stilig, underholdende og sofistikert!: 
...  på. 

Gammeldags kvinnesyn 

Det som vil more og/eller provosere flest i ”Mad Men” er kvinnesynet serien skildrer. 
Her er mannen den absolutte sjefen, og kvinnen er til for å tjene mannen. 
Seriens  kvinner er  inneforstått med det, men er samtidig oppmerksomme på den kraften og innflytelsen de kan ha på menn. 
Det finnes kvinner i serien som bruker menns kvinnesyn til egen fordel. 

Samtidig er det interes


  7%|▋         | 2501/34478 [00:00<00:09, 3376.62it/s]

---
Den tyske filmen ”Den frie vilje” sitter som et knyttneveslag midt i magen.: 
... 

Det er lett å forstå hvilken vei det bærer. 
Men til tross for det, håper vi for Theo. 
Filmen forteller ingenting om hans bakgrunn, men vi skjønner at han må være fryktelig ødelagt. 

Hans hat til  kvinner er  like intenst som det er uforståelig. 
Men jeg liker at filmen ikke overforklarer. 
Vi må selv finne ut av Theos personlighet. 
Og vi må selv ta stilling til hvorvidt dette er en person vi vil føle sy
---
fint romantisk drama: 
... sisk samlivsbrudd. 
De står foran mølla alle skilte må igjennom, men spørsmålet er om det er dette de egentlig vil. 
Og kan bruddet leges? 

Filmens tittel spiller selvsagt på boka ”Menn er fra Mars,  kvinner er  fra Venus” som forteller om menn og kvinners ulike behov. 
Du får nok ingen dyptpløyende psykoanalyser i denne filmen, men tekster fra boka, lest av parets sønn, er med på å forklare reaksjonsmønstre
---
Elegant og gjennomført: 
... Mannssjåvinistisk kvinne

 13%|█▎        | 4652/34478 [00:01<00:09, 3196.09it/s]

---
Langtekkelig oppfølger til langtekkelig forgjenger.: 
... l. 

«Luddere» og «gangstere», velkommen til «Frank Miller’s Sin City». 
(Våg for all del ikke å kalle den noe kortere og greiere enn «FRANK MILLER’S Sin City» – da blir nemlig «skaperen» vred). 
Der  kvinner er  sørgende strippere og/eller troløse horer, og menn er infantile, voldelige tosker. 

Vi har vært her før, som noen vil huske, i den like monotone og monokrome forgjengeren, «Sin City» – unnskyld! 
«F


 15%|█▍        | 5131/34478 [00:01<00:10, 2811.57it/s]

---
: 
... igjen treffer på en stadig mer nedkjørt Harry Hole. 
Han må hentes ut av opiumsrusen i Hong Kong, for ytterst motvillig å bistå Oslo-politiet med det de frykter er en ny seriemorder på gang. 
To unge  kvinner er  funnet drept med bedøvelsesmiddel i blodet og uforklarlige stikksår i munnen. 
Det skal vise seg at de er kommet i nærkontakt med det utspekulert sadistiske drapsvåpenet «Leopolds eple». 
Og ikke så 
---
Found max occurences, stopped before searching through entire dataset


[3, 3219, 3221, 3463, 3506, 3826, 3831, 3940, 101437, 102139]